In [2]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
import pickle
from sklearn.metrics.pairwise import cosine_similarity
# from haversine import haversine, Unit

/home/ethan/anaconda3/envs/pl_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("../data/prep3.csv", encoding='cp949')

In [5]:
pwd

'/home/ethan/git_rp/git/LLM/script'

# API Key & Base URL:A.company 5070TI

In [6]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

# Embedding Function

In [7]:
class OllamaSentenceTransformer():
    def __init__(
            self,
            *args,
            **kargs,
            ) -> None:
                self.base_url = kargs.get("base_url", "http://localhost:11434")
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            base_url = self.base_url,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)
        


In [16]:
sentence_transformer = OllamaSentenceTransformer(base_url=base_url)

In [18]:
sentence_transformer.encode(['123','24','234'])

array([[-0.02764925,  0.00978895, -0.03736658, ..., -0.01262593,
         0.01433419,  0.0590205 ],
       [-0.0051665 ,  0.04774755, -0.02180332, ..., -0.0344875 ,
        -0.00380494,  0.05704403],
       [-0.03296679,  0.05047558, -0.05491041, ...,  0.01124008,
         0.04521679,  0.10267203]])

# Input Content

In [19]:
with open('../data/user_input_content.json', "rb") as f:
    input_content = json.load(f)

input_content

{'tool_name': '전동드릴',
 'location': '강남역',
 'job_content': '전동드릴을 사용해서 나무를 고정하고 싶어'}

# Question

In [22]:
question = f"""나는 {input_content['tool_name']}을(를) 빌리고 싶어. 
그리고 이 도구를 이용해서 하려는 작업은 '{input_content['job_content']}' 이야. 
해당 작업을 하면서 같이 빌리면 좋은 공구도 함께 알려줘"""

# LLM Define
## Ver.OpenAI

In [ ]:
def invoke(question:str,
           history:str = None, 
           api_key:str=api_key, 
           model:str="gpt-4o", 
           temperature:float=0.7):
    client = OpenAI(api_key=api_key)
    
    system_content = (
        """너는 사람들의 질문에 친절히 답해주는 도우미야.
            사용자의 질문에 친절히 답해줘."""
    )
    messages = [{"role": "system", "content": system_content}]
    
    # 이전 대화 히스토리를 포함
    if history:
        messages.extend(history)

    # 마지막 질문 추가
    messages.append({"role": "user", "content": question})
    
    
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    return stream
    

In [33]:
system_content = (
        """너는 사람들의 질문에 친절히 답해주는 도우미야.
            사용자의 질문에 친절히 답해줘."""
    )
messages = [{"role": "system", "content": system_content}]

messages.extend([{"role": "user", "content": 'user_input'},
                 {"role": "system", "content": 'response_text'}
                 ])


In [34]:
messages

[{'role': 'system',
  'content': '너는 사람들의 질문에 친절히 답해주는 도우미야.\n            사용자의 질문에 친절히 답해줘.'},
 {'role': 'user', 'content': 'user_input'},
 {'role': 'system', 'content': 'response_text'}]

### Output the response

In [ ]:
for chunk in invoke(question):
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='', flush=True)

### Response based History

In [ ]:
history = []

while True:
    user_input = input("\n🙂 질문을 입력하세요: ")
    if user_input.lower() in ["exit", "quit", "종료", "끝"]:
        break

    print("🧠 GPT 응답:")
    response_text = ""
    
    for chunk in invoke(user_input, history=history, api_key=api_key):
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="", flush=True)
            response_text += delta.content

    # 응답 후 대화 이력 업데이트
    history.append({"role": "user", "content": user_input})
    history.append({"role": "system", "content": response_text})

## Ver.Gemma3

In [ ]:
from langchain_ollama.chat_models import ChatOllama

def invoke(question, info = None, model="gemma3:12b", temperature=0.7):
    chat = ChatOllama(model=model,
                      base_url=base_url,
                      temperature=temperature,)

    system_content = (
                """너는 사람들의 질문에 친절히 답해주는 도우미야.
                    사용자의 질문에 친절히 답해줘."""
        )

    messages = [{"role": "system", "content": system_content}]
    
    # 이전 대화 히스토리를 포함
    if history:
        messages.extend(history)

    # 마지막 질문 추가
    messages.append({"role": "user", "content": question})
    

    # invoke로 전체 응답 받기
    # response = chat.invoke(messages).content
    response = chat.stream(messages)

    # 응답 내용 반환
    return response


### Output the response

In [ ]:
for chunk in invoke(question):
    print(chunk.content, end='', flush=True)

In [ ]:
history = []

while True:
    user_input = input("\n🙂 질문을 입력하세요: ")
    if user_input.lower() in ["exit", "quit", "종료", "끝"]:
        break

    print("🧠 GPT 응답:")
    response_text = ""
    
    for chunk in invoke(user_input, history=history, api_key=api_key):
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="", flush=True)
            response_text += delta.content

    # 응답 후 대화 이력 업데이트
    history.append({"role": "user", "content": user_input})
    history.append({"role": "system", "content": response_text})